In [5]:

## velocity maps CSK and TSX per period

"""
indexes of unique dates for date range
CSK: 
period 1: (calm before the storm) 0 -> 5 (2020-09-10 -> 2020-10-13) [2 cm/day]
period 2: (the storm pt. 1) 5 -> 13 (2020-10-13 -> 2021-01-09) [20 cm/day]
period 3: (the eye of the storm) 13 -> 37 (2021-01-09 -> 2021-07-11) [3 cm/day]
period 4: (the storm pt. 2) 37 -> 42 (2021-07-11 -> 2021-08-13) [15 cm/day]
period 5: (the calm after the storm) 42 -> 54 (2021-08-13 -> 2021-11-01) [2 cm/day]


TSX:
period 1: (calm before the storm) no data
period 2: (the storm pt. 1) 0 -> 4 (2020-11-16 -> 2021-01-10) [20 cm/day]
period 3: (the eye of the storm) 4 -> 12 (2021-01-10 -> 2021-06-13) [3 cm/day]
period 4: (the storm pt. 2) 12 -> 13 (2021-06-13 -> 2021-10-01) [15 cm/day]
period 5: (the calm after the storm) 13 -> 14 (2021-10-01 -> 2021-10-23) [2 cm/day]
"""

# steps

"""
step 1: load dependencies
step 2: load data
step 3: seperate data per time period
step 4: visualise range offset rate, azimuth offset rate, speed in range-azimuth plane

"""


'\nstep 1: load dependencies\nstep 2: load data\nstep 3: seperate data per time period\nstep 4: visualise range offset rate, azimuth offset rate, speed in range-azimuth plane\n\n'

In [6]:
## first time series test
# append ../../ to make link to main repo work
import sys
sys.path.append('../../')

# import external packages
import numpy as np
import pandas as pd
import numba
from numba import vectorize
import glob # for file search
import copy
import os # operating system stuff
import re # regex


import geopandas as gpd
import rasterio as rio
import rasterio.mask
from rasterio.plot import plotting_extent
from shapely.geometry import Polygon
from shapely.geometry.point import Point
import pyproj
from pyproj import CRS
from inpoly import inpoly2 # for fast inpolygon checks
import utm

import matplotlib.pyplot as plt 
import matplotlib.dates as mdates
from matplotlib import cm as mpl_cm
from matplotlib import colors as mcolors 
import matplotlib.image as mplimg

from mpl_toolkits.axes_grid1 import make_axes_locatable # for colorbar scaling
from mpl_toolkits.axes_grid1 import ImageGrid
from matplotlib_scalebar.scalebar import ScaleBar
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import FormatStrFormatter

import seaborn as sns
from matplotlib import rc_file_defaults
rc_file_defaults()
# sns.set(style=None, color_codes=True)

from shapely.geometry import Polygon
from shapely.geometry.point import Point
import datetime

import configparser

from cmcrameri import cm # for scientific colourmaps

###########################
# import main local package
import SPOTSAR_main as sm


In [7]:
# load data

# load SPOT time-series data
# CSK
CSK_TS_FILE = '/Users/tz20896/Documents/PhD/projects/Merapi2021/CSK/dsc1/CSK_SPOT_dsc_win13_ts_inv.csv'
CSK_TS = pd.read_csv(CSK_TS_FILE)
# TSX
TSX_TS_FILE = '/Users/tz20896/Documents/PhD/projects/Merapi2021/TSX/staring_134/TSX_ST134_win13_ts_inv.csv'
TSX_TS = pd.read_csv(TSX_TS_FILE)

# show TS data

print(CSK_TS)
print(TSX_TS)

CSK_TS_new = CSK_TS.iloc[:,2:]
print(CSK_TS_new)
# CSK_TS_new.to_csv(CSK_TS_FILE)

# lava flow files:
L1888_FILE  = '/Users/tz20896/Documents/PhD/projects/Merapi2021/merapi_maps/L1888_v2.shp'
L1948_FILE  = '/Users/tz20896/Documents/PhD/projects/Merapi2021/merapi_maps/L1948.shp'
L1956_FILE  = '/Users/tz20896/Documents/PhD/projects/Merapi2021/merapi_maps/L1956.shp'
L1992_FILE  = '/Users/tz20896/Documents/PhD/projects/Merapi2021/merapi_maps/L1992.shp'
L1997_FILE  = '/Users/tz20896/Documents/PhD/projects/Merapi2021/merapi_maps/L1997.shp'
L1998_FILE  = '/Users/tz20896/Documents/PhD/projects/Merapi2021/merapi_maps/L1998.shp'
CRATER_FILE = '/Users/tz20896/Documents/PhD/projects/Merapi2021/merapi_maps/Merapi_crater.shp'

# Read the shapefile
L1888  = gpd.read_file(L1888_FILE)
L1948  = gpd.read_file(L1948_FILE)
L1956  = gpd.read_file(L1956_FILE)
L1992  = gpd.read_file(L1992_FILE)
L1997  = gpd.read_file(L1997_FILE)
L1998  = gpd.read_file(L1998_FILE)
CRATER = gpd.read_file(CRATER_FILE)

# Extract latitude and longitude into separate columns
coords_L1888   = np.array(list(L1888["geometry"][0].coords))
coords_L1956_1 = np.array(list(L1956["geometry"][0].coords))
coords_L1956_2 = np.array(list(L1956["geometry"][1].coords))
coords_L1948   = np.array(list(L1948["geometry"][0].coords))
coords_L1992   = np.array(list(L1992["geometry"][0].coords))
coords_L1997   = np.array(list(L1997["geometry"][0].coords))
coords_L1998   = np.array(list(L1998["geometry"][0].coords))
coords_CRATER  = np.array(list(CRATER["geometry"][0].coords))




        Unnamed: 0   Longitude  Latitude  r_disp_  a_disp_  r_disp_20200910  \
0                0  110.483566 -7.519673      0.0      0.0        -0.081862   
1                1  110.483627 -7.519764      0.0      0.0        -0.110186   
2                2  110.483574 -7.519828      0.0      0.0        -0.116883   
3                3  110.483536 -7.519897      0.0      0.0        -0.141539   
4                4  110.483505 -7.519970      0.0      0.0        -0.152580   
...            ...         ...       ...      ...      ...              ...   
282265      310758  110.418274 -7.561081      0.0      0.0         0.012227   
282266      310761  110.427368 -7.509089      0.0      0.0        -0.021778   
282267      310764  110.418236 -7.561071      0.0      0.0         0.036422   
282268      310765  110.418282 -7.561155      0.0      0.0         0.046467   
282269      310775  110.418449 -7.560341      0.0      0.0         0.040927   

        a_disp_20200910  r_disp_20200919  a_disp_20

In [9]:
### get minmimum and maximum lon and lat of CSK and TSX data

# CSK data

lon_min_id = np.argmin(CSK_TS['Longitude'])
lon_max_id = np.argmax(CSK_TS['Longitude'])
lat_min_id = np.argmin(CSK_TS['Latitude'])
lat_max_id = np.argmax(CSK_TS['Latitude'])

# print corners
print(CSK_TS['Longitude'][lon_min_id], CSK_TS['Latitude'][lon_min_id]) # lower left corner
print(CSK_TS['Longitude'][lat_min_id], CSK_TS['Latitude'][lat_min_id]) # lower right corner
print(CSK_TS['Longitude'][lon_max_id], CSK_TS['Latitude'][lon_max_id]) # upper right corner
print(CSK_TS['Longitude'][lat_max_id], CSK_TS['Latitude'][lat_max_id]) # upper left corner

# TSX data

lon_min_id = np.argmin(TSX_TS['Longitude'])
lon_max_id = np.argmax(TSX_TS['Longitude'])
lat_min_id = np.argmin(TSX_TS['Latitude'])
lat_max_id = np.argmax(TSX_TS['Latitude'])

# print corners
print(TSX_TS['Longitude'][lon_min_id], TSX_TS['Latitude'][lon_min_id]) # lower left corner
print(TSX_TS['Longitude'][lat_min_id], TSX_TS['Latitude'][lat_min_id]) # lower right corner
print(TSX_TS['Longitude'][lon_max_id], TSX_TS['Latitude'][lon_max_id]) # upper right corner
print(TSX_TS['Longitude'][lat_max_id], TSX_TS['Latitude'][lat_max_id]) # upper left corner

110.41771697998048 -7.563600540161133
110.4740753173828 -7.575775146484375
110.48404693603516 -7.520786285400391
110.42781829833984 -7.507633686065674
110.42473602294922 -7.538825035095215
110.46287536621094 -7.547752380371094
110.46599578857422 -7.540246963500977
110.42674255371094 -7.528644561767578


In [4]:
# CSK_TS_new.to_csv(CSK_TS_FILE)
# # print(CSK_TS)

In [10]:
## separate data per time period:

col_offset = 3
CSK_cols_r = list(CSK_TS)[col_offset+2::2]
TSX_cols_r = list(TSX_TS)[col_offset::2]

CSK_dates = [datetime.datetime.strptime(CSK_col_r[-8:],'%Y%m%d') for CSK_col_r in CSK_cols_r]
TSX_dates = [datetime.datetime.strptime(TSX_col_r[-8:],'%Y%m%d') for TSX_col_r in TSX_cols_r]
CSK_dates.append(datetime.datetime.strptime('20211101','%Y%m%d') )
print(CSK_dates)
print(TSX_dates)

# CSK
# indexes of unique dates for date range
# CSK: 
# period 1: (calm before the storm) 0 -> 5 (2020-09-10 -> 2020-10-13) [2 cm/day]
# period 2: (the storm pt. 1) 5 -> 13 (2020-10-13 -> 2021-01-09) [20 cm/day]
# period 3: (the eye of the storm) 13 -> 37 (2021-01-09 -> 2021-07-11) [3 cm/day]
# period 4: (the storm pt. 2) 37 -> 42 (2021-07-11 -> 2021-08-13) [15 cm/day]
# period 5: (the calm after the storm) 42 -> 54 (2021-08-13 -> 2021-11-01) [2 cm/day]



CSK_delta_t_1 = CSK_dates[5] - CSK_dates[0]
CSK_delta_t_2 = CSK_dates[13] - CSK_dates[5]
CSK_delta_t_3 = CSK_dates[37] - CSK_dates[13]
CSK_delta_t_4 = CSK_dates[42] - CSK_dates[37]
CSK_delta_t_5 = CSK_dates[54] - CSK_dates[42]
CSK_delta_t_total = CSK_dates[-1] - CSK_dates[0]

print(CSK_delta_t_1)

CSK_R_disp_rate_1 = (CSK_TS.iloc[:,col_offset+5 *2]  - CSK_TS.iloc[:,col_offset+0*2])/CSK_delta_t_1.days
CSK_R_disp_rate_2 = (CSK_TS.iloc[:,col_offset+13*2] - CSK_TS.iloc[:,col_offset +5*2])/CSK_delta_t_2.days
CSK_R_disp_rate_3 = (CSK_TS.iloc[:,col_offset+37*2] - CSK_TS.iloc[:,col_offset+13*2])/CSK_delta_t_3.days
CSK_R_disp_rate_4 = (CSK_TS.iloc[:,col_offset+42*2] - CSK_TS.iloc[:,col_offset+37*2])/CSK_delta_t_4.days
CSK_R_disp_rate_5 = (CSK_TS.iloc[:,col_offset+54*2] - CSK_TS.iloc[:,col_offset+42*2])/CSK_delta_t_5.days

CSK_A_disp_rate_1 = (CSK_TS.iloc[:,col_offset +5*2 +1]  - CSK_TS.iloc[:,col_offset+0*2 +1])/CSK_delta_t_1.days
CSK_A_disp_rate_2 = (CSK_TS.iloc[:,col_offset+13*2 +1] - CSK_TS.iloc[:,col_offset +5*2 +1])/CSK_delta_t_2.days
CSK_A_disp_rate_3 = (CSK_TS.iloc[:,col_offset+37*2 +1] - CSK_TS.iloc[:,col_offset+13*2 +1])/CSK_delta_t_3.days
CSK_A_disp_rate_4 = (CSK_TS.iloc[:,col_offset+42*2 +1] - CSK_TS.iloc[:,col_offset+37*2 +1])/CSK_delta_t_4.days
CSK_A_disp_rate_5 = (CSK_TS.iloc[:,col_offset+54*2 +1] - CSK_TS.iloc[:,col_offset+42*2 +1])/CSK_delta_t_5.days


CSK_R_disp_rate_total = (CSK_TS.iloc[:,-2] - CSK_TS.iloc[:,col_offset])/CSK_delta_t_total.days
CSK_A_disp_rate_total = (CSK_TS.iloc[:,-1] - CSK_TS.iloc[:,col_offset+1])/CSK_delta_t_total.days



# TSX:
# period 1: (calm before the storm) no data
# period 2: (the storm pt. 1) 0 -> 4 (2020-11-16 -> 2021-01-10) [20 cm/day]
# period 3: (the eye of the storm) 4 -> 12 (2021-01-10 -> 2021-06-13) [3 cm/day]
# period 4: (the storm pt. 2) 12 -> 13 (2021-06-13 -> 2021-10-01) [15 cm/day]
# period 5: (the calm after the storm) 13 -> 14 (2021-10-01 -> 2021-10-23) [2 cm/day]

TSX_delta_t_2 = TSX_dates[4] - TSX_dates[0]
TSX_delta_t_3 = TSX_dates[12] - TSX_dates[4]
TSX_delta_t_4 = TSX_dates[13] - TSX_dates[12]
TSX_delta_t_5 = TSX_dates[14] - TSX_dates[13]
TSX_delta_t_total = TSX_dates[14] - TSX_dates[0]

TSX_R_disp_rate_2 = (TSX_TS.iloc[:,col_offset+4*2] - TSX_TS.iloc[:,col_offset +0*2]) /TSX_delta_t_2.days
TSX_R_disp_rate_3 = (TSX_TS.iloc[:,col_offset+12*2] - TSX_TS.iloc[:,col_offset+4*2]) /TSX_delta_t_3.days
TSX_R_disp_rate_4 = (TSX_TS.iloc[:,col_offset+13*2] - TSX_TS.iloc[:,col_offset+12*2])/TSX_delta_t_4.days
TSX_R_disp_rate_5 = (TSX_TS.iloc[:,col_offset+14*2] - TSX_TS.iloc[:,col_offset+13*2])/TSX_delta_t_5.days


TSX_A_disp_rate_2 = (TSX_TS.iloc[:,col_offset+4*2 +1] - TSX_TS.iloc[:,col_offset +0*2 +1])/TSX_delta_t_2.days
TSX_A_disp_rate_3 = (TSX_TS.iloc[:,col_offset+12*2 +1] - TSX_TS.iloc[:,col_offset+4*2 +1])/TSX_delta_t_3.days
TSX_A_disp_rate_4 = (TSX_TS.iloc[:,col_offset+13*2 +1] - TSX_TS.iloc[:,col_offset+12*2 +1])/TSX_delta_t_4.days
TSX_A_disp_rate_5 = (TSX_TS.iloc[:,col_offset+14*2 +1] - TSX_TS.iloc[:,col_offset+13*2 +1])/TSX_delta_t_5.days


TSX_R_disp_rate_total = (TSX_TS.iloc[:,-2] - TSX_TS.iloc[:,col_offset])/TSX_delta_t_total.days
TSX_A_disp_rate_total = (TSX_TS.iloc[:,-1] - TSX_TS.iloc[:,col_offset+1])/TSX_delta_t_total.days


[datetime.datetime(2020, 9, 10, 0, 0), datetime.datetime(2020, 9, 19, 0, 0), datetime.datetime(2020, 9, 26, 0, 0), datetime.datetime(2020, 9, 27, 0, 0), datetime.datetime(2020, 10, 5, 0, 0), datetime.datetime(2020, 10, 12, 0, 0), datetime.datetime(2020, 10, 13, 0, 0), datetime.datetime(2020, 11, 13, 0, 0), datetime.datetime(2020, 11, 14, 0, 0), datetime.datetime(2020, 11, 22, 0, 0), datetime.datetime(2020, 12, 24, 0, 0), datetime.datetime(2020, 12, 31, 0, 0), datetime.datetime(2021, 1, 1, 0, 0), datetime.datetime(2021, 1, 9, 0, 0), datetime.datetime(2021, 1, 16, 0, 0), datetime.datetime(2021, 1, 17, 0, 0), datetime.datetime(2021, 1, 25, 0, 0), datetime.datetime(2021, 2, 1, 0, 0), datetime.datetime(2021, 2, 2, 0, 0), datetime.datetime(2021, 2, 17, 0, 0), datetime.datetime(2021, 2, 18, 0, 0), datetime.datetime(2021, 3, 2, 0, 0), datetime.datetime(2021, 3, 5, 0, 0), datetime.datetime(2021, 3, 6, 0, 0), datetime.datetime(2021, 3, 14, 0, 0), datetime.datetime(2021, 3, 21, 0, 0), datetime.da

In [6]:
import matplotlib.patches as mpatches
import matplotlib.transforms as mtransforms
def add_bottom_cax(ax, pad, height):
    axpos = ax.get_position()
    caxpos = mtransforms.Bbox.from_extents(
        axpos.x0,
        axpos.y0 - pad - height,
        axpos.x1,
        axpos.y0 - pad
    )
    cax = ax.figure.add_axes(caxpos)

    return cax

In [15]:
%matplotlib osx

plt.close('all')
grid_size= 1000
vmax = 1
cmap = cm.vik
col_offset=3
fig, ax = plt.subplots(1,1,figsize=(8,8))

plot_data = ax.hexbin(CSK_TS['Longitude'],CSK_TS['Latitude'],C=CSK_TS.iloc[:,col_offset+7*2]-CSK_TS.iloc[:,col_offset+6*2],gridsize=grid_size,cmap=cmap,vmin=-vmax,vmax=vmax)



: 

In [14]:
# visualise results

%matplotlib osx
# display image
import string
plt.close('all')

fig = plt.figure(figsize=(15,8))

gs = GridSpec(2, 5, figure=fig)

ax00 = fig.add_subplot(gs[0, 0]) # CSK P1
ax01 = fig.add_subplot(gs[0, 1]) # CSK P2
ax02 = fig.add_subplot(gs[0, 2]) # CSK P3
ax03 = fig.add_subplot(gs[0, 3]) # CSK P4
ax04 = fig.add_subplot(gs[0, 4]) # CSK P5


ax10 = fig.add_subplot(gs[1, 0]) # legend?
ax11 = fig.add_subplot(gs[1, 1]) # TSX P2
ax12 = fig.add_subplot(gs[1, 2]) # TSX P3
ax13 = fig.add_subplot(gs[1, 3]) # TSX P4
ax14 = fig.add_subplot(gs[1, 4]) # TSX P5

axes = [ax00,ax01,ax02,ax03,ax04,ax11,ax12,ax13,ax14]

data_to_plot_R = [CSK_R_disp_rate_1,
                CSK_R_disp_rate_2,
                CSK_R_disp_rate_3,
                CSK_R_disp_rate_4,
                CSK_R_disp_rate_5,
                TSX_R_disp_rate_2,
                TSX_R_disp_rate_3, 
                TSX_R_disp_rate_4, 
                TSX_R_disp_rate_5]

data_to_plot_A = [CSK_A_disp_rate_1,
                CSK_A_disp_rate_2,
                CSK_A_disp_rate_3,
                CSK_A_disp_rate_4,
                CSK_A_disp_rate_5,
                TSX_A_disp_rate_2,
                TSX_A_disp_rate_3, 
                TSX_A_disp_rate_4, 
                TSX_A_disp_rate_5]

Lons = [CSK_TS['Longitude'],
        CSK_TS['Longitude'],
        CSK_TS['Longitude'],
        CSK_TS['Longitude'],
        CSK_TS['Longitude'],
        TSX_TS['Longitude'],
        TSX_TS['Longitude'], 
        TSX_TS['Longitude'], 
        TSX_TS['Longitude']]


Lats = [CSK_TS['Latitude'],
        CSK_TS['Latitude'],
        CSK_TS['Latitude'],
        CSK_TS['Latitude'],
        CSK_TS['Latitude'],
        TSX_TS['Latitude'],
        TSX_TS['Latitude'], 
        TSX_TS['Latitude'], 
        TSX_TS['Latitude']]


SCALE = 100 # cm/m

clims = [(-2,2),
        (-20,20),
        (-3,3),
        (-15,15),
        (-2,2),
        (-20,20),
        (-3,3),
        (-10,10),
        (-2,2)]

clims = [(-0.5,0.5),
        (-15,15),
        (-5,5),
        (-10,10),
        (-2,2),
        (-15,15),
        (-5,5),
        (-10,10),
        (-2,2)]


# clims = [(-3,3),
#          (-3,3),
#          (-3,3),
#          (-3,3),
#          (-3,3),
#          (-3,3),
#          (-3,3),
#          (-3,3),
#          (-3,3)]

delta_T = [CSK_delta_t_1.days,
           CSK_delta_t_2.days,
           CSK_delta_t_3.days,
           CSK_delta_t_4.days,
           CSK_delta_t_5.days,
           TSX_delta_t_2.days,
           TSX_delta_t_3.days,
           TSX_delta_t_4.days,
           TSX_delta_t_5.days]

cmaps = [cm.vik, cm.vik, cm.vik, cm.vik, cm.vik,
         cm.vik, cm.vik, cm.vik, cm.vik]

labels = string.ascii_uppercase[0:len(axes)]

label_x = [0.03,0.03,0.03,0.03,0.03,
           0.03,0.03,0.03,0.03]
label_y = [0.86,0.86,0.86,0.86,0.86,
           0.86,0.86,0.86,0.86,]

lon_lims = [110.432, 110.45]
lat_lims = [-7.547, -7.535]
grid_size = 1000
disp = 1
minmax = 0
bbox = dict(fc="1.0",alpha=0.5)
for (a,data,lon,lat,label,lab_x,lab_y,cmap,clim,t_diff) in zip(axes,data_to_plot_R,Lons,Lats,labels,label_x,label_y,cmaps,clims,delta_T):
    if disp:
        if minmax:
            vmax = np.max(np.abs(data*t_diff))
            print(vmax)
            plot_data = a.hexbin(lon,lat,C=data*t_diff,gridsize=grid_size,cmap=cmap,vmin=-vmax,vmax=vmax)
        else:
            plot_data = a.hexbin(lon,lat,C=data*t_diff,gridsize=grid_size,cmap=cmap,vmin=clim[0],vmax=clim[1])
    else:
        if minmax:
            vmax = np.max(np.abs(data*SCALE))
            print(vmax)
            plot_data = a.hexbin(lon,lat,C=data*SCALE,gridsize=grid_size,cmap=cmap,vmin=-vmax,vmax=vmax)
        else:
            plot_data = a.hexbin(lon,lat,C=data*SCALE,gridsize=grid_size,cmap=cmap,vmin=clim[0],vmax=clim[1])
    a.plot(coords_L1888[:,0],coords_L1888[:,1],linewidth=2,color='tab:blue',label='L1888')
    a.plot(coords_L1948[:,0],coords_L1948[:,1],linewidth=2,color='tab:cyan',label='L1948')
    a.plot(coords_L1956_1[:,0],coords_L1956_1[:,1],linewidth=2,color='tab:red',label='L1956')
    a.plot(coords_L1956_2[:,0],coords_L1956_2[:,1],linewidth=2,color='tab:red')
    a.plot(coords_L1992[:,0],coords_L1992[:,1],linewidth=2,color='tab:purple',label='L1992')
    a.plot(coords_L1997[:,0],coords_L1997[:,1],linewidth=2,color='tab:olive',label='L1997')
    a.plot(coords_L1998[:,0],coords_L1998[:,1],linewidth=2,color='tab:green',label='L1998')
    a.plot(coords_CRATER[:,0],coords_CRATER[:,1],linewidth=2,color='black',linestyle='--',label='Crater rim')
    a.set_xlim(lon_lims)
    a.set_ylim(lat_lims)
    a.set_aspect('equal', 'box')
    a.add_artist(ScaleBar(sm.plot.get_1deg_dist(),location='lower left'))
    # a.imshow(np.fliplr(np.rot90(image_crop,3)),cmap='Greys_r')
    a.annotate(label,xy=[lab_x,lab_y],xycoords='axes fraction',fontsize=16, bbox=bbox)
    # a.annotate(date.strftime('%Y-%m-%d'),xy=[lab_x+0.0,lab_y-0.795],xycoords='axes fraction',fontsize=16, bbox=bbox)
    a.set_axis_off()
ax10.set_axis_off()
fig.subplots_adjust(wspace=0.01,hspace=0.01)




In [ ]:
# visualise results

%matplotlib osx
# display image
import string
plt.close('all')

fig = plt.figure(figsize=(15,8))

gs = GridSpec(2, 5, figure=fig)

ax00 = fig.add_subplot(gs[0, 0]) # CSK P1
ax01 = fig.add_subplot(gs[0, 1]) # CSK P2
ax02 = fig.add_subplot(gs[0, 2]) # CSK P3
ax03 = fig.add_subplot(gs[0, 3]) # CSK P4
ax04 = fig.add_subplot(gs[0, 4]) # CSK P5


ax10 = fig.add_subplot(gs[1, 0]) # legend?
ax11 = fig.add_subplot(gs[1, 1]) # TSX P2
ax12 = fig.add_subplot(gs[1, 2]) # TSX P3
ax13 = fig.add_subplot(gs[1, 3]) # TSX P4
ax14 = fig.add_subplot(gs[1, 4]) # TSX P5

axes = [ax00,ax01,ax02,ax03,ax04,ax11,ax12,ax13,ax14]

data_to_plot_R = [CSK_R_disp_rate_1,
                CSK_R_disp_rate_2,
                CSK_R_disp_rate_3,
                CSK_R_disp_rate_4,
                CSK_R_disp_rate_5,
                TSX_R_disp_rate_2,
                TSX_R_disp_rate_3, 
                TSX_R_disp_rate_4, 
                TSX_R_disp_rate_5]

data_to_plot_A = [CSK_A_disp_rate_1,
                CSK_A_disp_rate_2,
                CSK_A_disp_rate_3,
                CSK_A_disp_rate_4,
                CSK_A_disp_rate_5,
                TSX_A_disp_rate_2,
                TSX_A_disp_rate_3, 
                TSX_A_disp_rate_4, 
                TSX_A_disp_rate_5]

Lons = [CSK_TS['Longitude'],
        CSK_TS['Longitude'],
        CSK_TS['Longitude'],
        CSK_TS['Longitude'],
        CSK_TS['Longitude'],
        TSX_TS['Longitude'],
        TSX_TS['Longitude'], 
        TSX_TS['Longitude'], 
        TSX_TS['Longitude']]


Lats = [CSK_TS['Latitude'],
        CSK_TS['Latitude'],
        CSK_TS['Latitude'],
        CSK_TS['Latitude'],
        CSK_TS['Latitude'],
        TSX_TS['Latitude'],
        TSX_TS['Latitude'], 
        TSX_TS['Latitude'], 
        TSX_TS['Latitude']]


SCALE = 100 # cm/m

clims = [(-2,2),
        (-20,20),
        (-3,3),
        (-15,15),
        (-2,2),
        (-20,20),
        (-3,3),
        (-10,10),
        (-2,2)]

clims = [(-0.5,0.5),
        (-15,15),
        (-5,5),
        (-10,10),
        (-2,2),
        (-15,15),
        (-5,5),
        (-10,10),
        (-2,2)]


clims = [(-3,3),
         (-3,3),
         (-3,3),
         (-3,3),
         (-3,3),
         (-3,3),
         (-3,3),
         (-3,3),
         (-3,3)]

delta_T = [CSK_delta_t_1.days,
           CSK_delta_t_2.days,
           CSK_delta_t_3.days,
           CSK_delta_t_4.days,
           CSK_delta_t_5.days,
           TSX_delta_t_2.days,
           TSX_delta_t_3.days,
           TSX_delta_t_4.days,
           TSX_delta_t_5.days]

cmaps = [cm.vik, cm.vik, cm.vik, cm.vik, cm.vik,
         cm.vik, cm.vik, cm.vik, cm.vik]

labels = string.ascii_uppercase[0:len(axes)]

label_x = [0.03,0.03,0.03,0.03,0.03,
           0.03,0.03,0.03,0.03]
label_y = [0.86,0.86,0.86,0.86,0.86,
           0.86,0.86,0.86,0.86,]

lon_lims = [110.432, 110.45]
lat_lims = [-7.547, -7.535]
grid_size = 1000
disp = 1
minmax = 0
bbox = dict(fc="1.0",alpha=0.5)
for (a,data,lon,lat,label,lab_x,lab_y,cmap,clim,t_diff) in zip(axes,data_to_plot_A,Lons,Lats,labels,label_x,label_y,cmaps,clims,delta_T):
    if disp:
        if minmax:
            vmax = np.max(np.abs(data*t_diff))
            print(vmax)
            plot_data = a.hexbin(lon,lat,C=data*t_diff,gridsize=grid_size,cmap=cmap,vmin=-vmax,vmax=vmax)
        else:
            plot_data = a.hexbin(lon,lat,C=data*t_diff,gridsize=grid_size,cmap=cmap,vmin=clim[0],vmax=clim[1])
    else:
        if minmax:
            vmax = np.max(np.abs(data*SCALE))
            print(vmax)
            plot_data = a.hexbin(lon,lat,C=data*SCALE,gridsize=grid_size,cmap=cmap,vmin=-vmax,vmax=vmax)
        else:
            plot_data = a.hexbin(lon,lat,C=data*SCALE,gridsize=grid_size,cmap=cmap,vmin=clim[0],vmax=clim[1])
    a.plot(coords_L1888[:,0],coords_L1888[:,1],linewidth=2,color='tab:blue',label='L1888')
    a.plot(coords_L1948[:,0],coords_L1948[:,1],linewidth=2,color='tab:cyan',label='L1948')
    a.plot(coords_L1956_1[:,0],coords_L1956_1[:,1],linewidth=2,color='tab:red',label='L1956')
    a.plot(coords_L1956_2[:,0],coords_L1956_2[:,1],linewidth=2,color='tab:red')
    a.plot(coords_L1992[:,0],coords_L1992[:,1],linewidth=2,color='tab:purple',label='L1992')
    a.plot(coords_L1997[:,0],coords_L1997[:,1],linewidth=2,color='tab:olive',label='L1997')
    a.plot(coords_L1998[:,0],coords_L1998[:,1],linewidth=2,color='tab:green',label='L1998')
    a.plot(coords_CRATER[:,0],coords_CRATER[:,1],linewidth=2,color='black',linestyle='--',label='Crater rim')
    a.set_xlim(lon_lims)
    a.set_ylim(lat_lims)
    a.set_aspect('equal', 'box')
    a.add_artist(ScaleBar(sm.plot.get_1deg_dist(),location='lower left'))
    # a.imshow(np.fliplr(np.rot90(image_crop,3)),cmap='Greys_r')
    a.annotate(label,xy=[lab_x,lab_y],xycoords='axes fraction',fontsize=16, bbox=bbox)
    # a.annotate(date.strftime('%Y-%m-%d'),xy=[lab_x+0.0,lab_y-0.795],xycoords='axes fraction',fontsize=16, bbox=bbox)
    a.set_axis_off()
ax10.set_axis_off()
fig.subplots_adjust(wspace=0.01,hspace=0.01)




In [12]:
# visualise results

%matplotlib osx
# display image
import string
plt.close('all')

fig = plt.figure(figsize=(15,8))

gs = GridSpec(2, 5, figure=fig)

ax00 = fig.add_subplot(gs[0, 0]) # CSK P1
ax01 = fig.add_subplot(gs[0, 1]) # CSK P2
ax02 = fig.add_subplot(gs[0, 2]) # CSK P3
ax03 = fig.add_subplot(gs[0, 3]) # CSK P4
ax04 = fig.add_subplot(gs[0, 4]) # CSK P5


ax10 = fig.add_subplot(gs[1, 0]) # legend?
ax11 = fig.add_subplot(gs[1, 1]) # TSX P2
ax12 = fig.add_subplot(gs[1, 2]) # TSX P3
ax13 = fig.add_subplot(gs[1, 3]) # TSX P4
ax14 = fig.add_subplot(gs[1, 4]) # TSX P5

axes = [ax00,ax01,ax02,ax03,ax04,ax11,ax12,ax13,ax14]

data_to_plot_R = [CSK_R_disp_rate_1,
                  CSK_R_disp_rate_2,
                  CSK_R_disp_rate_3,
                  CSK_R_disp_rate_4,
                  CSK_R_disp_rate_5,
                  TSX_R_disp_rate_2,
                  TSX_R_disp_rate_3, 
                  TSX_R_disp_rate_4, 
                  TSX_R_disp_rate_5]

data_to_plot_A = [CSK_A_disp_rate_1,
                CSK_A_disp_rate_2,
                CSK_A_disp_rate_3,
                CSK_A_disp_rate_4,
                CSK_A_disp_rate_5,
                TSX_A_disp_rate_2,
                TSX_A_disp_rate_3, 
                TSX_A_disp_rate_4, 
                TSX_A_disp_rate_5]

data_to_plot_Mag = [np.hypot(CSK_A_disp_rate_1,CSK_R_disp_rate_1),
                    np.hypot(CSK_A_disp_rate_2,CSK_R_disp_rate_2),
                    np.hypot(CSK_A_disp_rate_3,CSK_R_disp_rate_3),
                    np.hypot(CSK_A_disp_rate_4,CSK_R_disp_rate_4),
                    np.hypot(CSK_A_disp_rate_5,CSK_R_disp_rate_5),
                    np.hypot(TSX_A_disp_rate_2,TSX_R_disp_rate_2),
                    np.hypot(TSX_A_disp_rate_3,TSX_R_disp_rate_3), 
                    np.hypot(TSX_A_disp_rate_4,TSX_R_disp_rate_4), 
                    np.hypot(TSX_A_disp_rate_5,TSX_R_disp_rate_5)]

Lons = [CSK_TS['Longitude'],
        CSK_TS['Longitude'],
        CSK_TS['Longitude'],
        CSK_TS['Longitude'],
        CSK_TS['Longitude'],
        TSX_TS['Longitude'],
        TSX_TS['Longitude'], 
        TSX_TS['Longitude'], 
        TSX_TS['Longitude']]


Lats = [CSK_TS['Latitude'],
        CSK_TS['Latitude'],
        CSK_TS['Latitude'],
        CSK_TS['Latitude'],
        CSK_TS['Latitude'],
        TSX_TS['Latitude'],
        TSX_TS['Latitude'], 
        TSX_TS['Latitude'], 
        TSX_TS['Latitude']]


SCALE = 100 # cm/m



clims = [(0,1),
         (0,1),
         (0,1),
         (0,1),
         (0,1),
         (0,1),
         (0,1),
         (0,1),
         (0,1)]

delta_T = [CSK_delta_t_1.days,
           CSK_delta_t_2.days,
           CSK_delta_t_3.days,
           CSK_delta_t_4.days,
           CSK_delta_t_5.days,
           TSX_delta_t_2.days,
           TSX_delta_t_3.days,
           TSX_delta_t_4.days,
           TSX_delta_t_5.days]

cmaps = ['viridis','viridis','viridis', 'viridis', 'viridis',
         'viridis','viridis','viridis', 'viridis']

labels = string.ascii_uppercase[0:len(axes)]

label_x = [0.03,0.03,0.03,0.03,0.03,
           0.03,0.03,0.03,0.03]
label_y = [0.86,0.86,0.86,0.86,0.86,
           0.86,0.86,0.86,0.86,]

lon_lims = [110.432, 110.45]
lat_lims = [-7.547, -7.535]
grid_size = 1000
disp = 0
minmax = 0
bbox = dict(fc="1.0",alpha=0.5)
for (a,data,lon,lat,label,lab_x,lab_y,cmap,clim,t_diff) in zip(axes,data_to_plot_Mag,Lons,Lats,labels,label_x,label_y,cmaps,clims,delta_T):
    if disp:
        if minmax:
            vmax = np.max(np.abs(data*t_diff))
            print(vmax)
            plot_data = a.hexbin(lon,lat,C=data*t_diff,gridsize=grid_size,cmap=cmap,vmin=-vmax,vmax=vmax)
        else:
            plot_data = a.hexbin(lon,lat,C=data*t_diff,gridsize=grid_size,cmap=cmap,vmin=clim[0],vmax=clim[1])
    else:
        if minmax:
            vmax = np.max(np.abs(data*SCALE))
            print(vmax)
            plot_data = a.hexbin(lon,lat,C=data*SCALE,gridsize=grid_size,cmap=cmap,vmin=-vmax,vmax=vmax)
        else:
            plot_data = a.hexbin(lon,lat,C=data*SCALE,gridsize=grid_size,cmap=cmap,vmin=clim[0],vmax=clim[1])
    a.plot(coords_L1888[:,0],coords_L1888[:,1],linewidth=2,color='tab:blue',label='L1888')
    a.plot(coords_L1948[:,0],coords_L1948[:,1],linewidth=2,color='tab:cyan',label='L1948')
    a.plot(coords_L1956_1[:,0],coords_L1956_1[:,1],linewidth=2,color='tab:red',label='L1956')
    a.plot(coords_L1956_2[:,0],coords_L1956_2[:,1],linewidth=2,color='tab:red')
    a.plot(coords_L1992[:,0],coords_L1992[:,1],linewidth=2,color='tab:purple',label='L1992')
    a.plot(coords_L1997[:,0],coords_L1997[:,1],linewidth=2,color='tab:olive',label='L1997')
    a.plot(coords_L1998[:,0],coords_L1998[:,1],linewidth=2,color='tab:green',label='L1998')
    a.plot(coords_CRATER[:,0],coords_CRATER[:,1],linewidth=2,color='black',linestyle='--',label='Crater rim')
    a.set_xlim(lon_lims)
    a.set_ylim(lat_lims)
    a.set_aspect('equal', 'box')
    a.add_artist(ScaleBar(sm.plot.get_1deg_dist(),location='lower left'))
    # a.imshow(np.fliplr(np.rot90(image_crop,3)),cmap='Greys_r')
    a.annotate(label,xy=[lab_x,lab_y],xycoords='axes fraction',fontsize=16, bbox=bbox)
    # a.annotate(date.strftime('%Y-%m-%d'),xy=[lab_x+0.0,lab_y-0.795],xycoords='axes fraction',fontsize=16, bbox=bbox)
    a.set_axis_off()
ax10.set_axis_off()
fig.subplots_adjust(wspace=0.01,hspace=0.01)




In [13]:
# visualise results

%matplotlib osx
# display image
import string
plt.close('all')

fig = plt.figure(figsize=(15,8))

gs = GridSpec(2, 3, figure=fig)

ax00 = fig.add_subplot(gs[0, 0]) # CSK P1
ax01 = fig.add_subplot(gs[0, 1]) # CSK P2
ax02 = fig.add_subplot(gs[0, 2]) # CSK P3
ax03 = fig.add_subplot(gs[1, 0]) # CSK P4
ax04 = fig.add_subplot(gs[1, 1]) # CSK P5
ax05 = fig.add_subplot(gs[1, 2]) # CSK P5


# ax10 = fig.add_subplot(gs[1, 0]) # legend?
# ax11 = fig.add_subplot(gs[1, 1]) # TSX P2
# ax12 = fig.add_subplot(gs[1, 2]) # TSX P3
# ax13 = fig.add_subplot(gs[1, 3]) # TSX P4
# ax14 = fig.add_subplot(gs[1, 4]) # TSX P5

axes = [ax00,ax01,ax02,ax03,ax04, ax05,ax11,ax12,ax13,ax14]

data_to_plot_R = [CSK_R_disp_rate_1,
                  CSK_R_disp_rate_2,
                  CSK_R_disp_rate_3,
                  CSK_R_disp_rate_4,
                  CSK_R_disp_rate_5,]

data_to_plot_A = [CSK_A_disp_rate_1,
                CSK_A_disp_rate_2,
                CSK_A_disp_rate_3,
                CSK_A_disp_rate_4,
                CSK_A_disp_rate_5,]

data_to_plot_Mag = [np.hypot(CSK_A_disp_rate_1,CSK_R_disp_rate_1),
                    np.hypot(CSK_A_disp_rate_2,CSK_R_disp_rate_2),
                    np.hypot(CSK_A_disp_rate_3,CSK_R_disp_rate_3),
                    np.hypot(CSK_A_disp_rate_4,CSK_R_disp_rate_4),
                    np.hypot(CSK_A_disp_rate_5,CSK_R_disp_rate_5),
                    np.hypot(CSK_A_disp_rate_total,CSK_R_disp_rate_total)]

Lons = [CSK_TS['Longitude'],
        CSK_TS['Longitude'],
        CSK_TS['Longitude'],
        CSK_TS['Longitude'],
        CSK_TS['Longitude'],
        CSK_TS['Longitude'],]


Lats = [CSK_TS['Latitude'],
        CSK_TS['Latitude'],
        CSK_TS['Latitude'],
        CSK_TS['Latitude'],
        CSK_TS['Latitude'],
        CSK_TS['Latitude'],]


SCALE = 100 # cm/m



clims = [(0,0.3),
         (0,15),
         (0,3),
         (0,5),
         (0,5),
         (0,1),]

clims = [(0,3),
         (0,3),
         (0,3),
         (0,3),
         (0,3),
         (0,3),]

delta_T = [CSK_delta_t_1.days,
           CSK_delta_t_2.days,
           CSK_delta_t_3.days,
           CSK_delta_t_4.days,
           CSK_delta_t_5.days,
           CSK_delta_t_total.days,]

cmaps = ['viridis','viridis','viridis', 'viridis', 'viridis', 'viridis',]

labels = string.ascii_uppercase[0:len(axes)]

label_x = [0.03,0.03,0.03,0.03,0.03,
           0.03,0.03,0.03,0.03]
label_y = [0.86,0.86,0.86,0.86,0.86,
           0.86,0.86,0.86,0.86,]

lon_lims = [110.432, 110.45]
lat_lims = [-7.547, -7.535]
grid_size = 1000
disp = 1
minmax = 0
bbox = dict(fc="1.0",alpha=0.5)
for (a,data,lon,lat,label,lab_x,lab_y,cmap,clim,t_diff) in zip(axes,data_to_plot_Mag[0:6],Lons,Lats,labels,label_x,label_y,cmaps,clims,delta_T):
    if disp:
        if minmax:
            vmax = np.max(np.abs(data*t_diff))
            print(vmax)
            plot_data = a.hexbin(lon,lat,C=data*t_diff,gridsize=grid_size,cmap=cmap,vmin=-vmax,vmax=vmax)
        else:
            plot_data = a.hexbin(lon,lat,C=data*t_diff,gridsize=grid_size,cmap=cmap,vmin=clim[0],vmax=clim[1])
    else:
        if minmax:
            vmax = np.max(np.abs(data*SCALE))
            print(vmax)
            plot_data = a.hexbin(lon,lat,C=data*SCALE,gridsize=grid_size,cmap=cmap,vmin=-vmax,vmax=vmax)
        else:
            plot_data = a.hexbin(lon,lat,C=data*SCALE,gridsize=grid_size,cmap=cmap,vmin=clim[0],vmax=clim[1])
    a.plot(coords_L1888[:,0],coords_L1888[:,1],linewidth=2,color='tab:blue',label='L1888')
    a.plot(coords_L1948[:,0],coords_L1948[:,1],linewidth=2,color='tab:cyan',label='L1948')
    a.plot(coords_L1956_1[:,0],coords_L1956_1[:,1],linewidth=2,color='tab:red',label='L1956')
    a.plot(coords_L1956_2[:,0],coords_L1956_2[:,1],linewidth=2,color='tab:red')
    a.plot(coords_L1992[:,0],coords_L1992[:,1],linewidth=2,color='tab:purple',label='L1992')
    a.plot(coords_L1997[:,0],coords_L1997[:,1],linewidth=2,color='tab:olive',label='L1997')
    a.plot(coords_L1998[:,0],coords_L1998[:,1],linewidth=2,color='tab:green',label='L1998')
    a.plot(coords_CRATER[:,0],coords_CRATER[:,1],linewidth=2,color='black',linestyle='--',label='Crater rim')
    a.set_xlim(lon_lims)
    a.set_ylim(lat_lims)
    a.set_aspect('equal', 'box')
    a.add_artist(ScaleBar(sm.plot.get_1deg_dist(),location='lower left'))
    # a.imshow(np.fliplr(np.rot90(image_crop,3)),cmap='Greys_r')
    a.annotate(label,xy=[lab_x,lab_y],xycoords='axes fraction',fontsize=16, bbox=bbox)
    # a.annotate(date.strftime('%Y-%m-%d'),xy=[lab_x+0.0,lab_y-0.795],xycoords='axes fraction',fontsize=16, bbox=bbox)
    cax = add_bottom_cax(a,0.04,0.02)
    cbar = fig.colorbar(plot_data,cax= cax, label='2D displacement magnitude [m]', orientation = 'horizontal', location='bottom')
    a.set_axis_off()
# ax10.set_axis_off()
ax04.legend(bbox_to_anchor=(1.05, 1),
                         loc='upper left', borderaxespad=0., frameon=False)

# fig.subplots_adjust(wspace=0.01,hspace=0.01)




In [ ]:
# visualise results

%matplotlib osx
# display image
import string
plt.close('all')

fig = plt.figure(figsize=(15,8))

gs = GridSpec(2, 3, figure=fig)

ax00 = fig.add_subplot(gs[0, 0]) # CSK P1
ax01 = fig.add_subplot(gs[0, 1]) # CSK P2
ax02 = fig.add_subplot(gs[0, 2]) # CSK P3
ax03 = fig.add_subplot(gs[1, 0]) # CSK P4
ax04 = fig.add_subplot(gs[1, 1]) # CSK P5
ax05 = fig.add_subplot(gs[1, 2]) # CSK P5


# ax10 = fig.add_subplot(gs[1, 0]) # legend?
# ax11 = fig.add_subplot(gs[1, 1]) # TSX P2
# ax12 = fig.add_subplot(gs[1, 2]) # TSX P3
# ax13 = fig.add_subplot(gs[1, 3]) # TSX P4
# ax14 = fig.add_subplot(gs[1, 4]) # TSX P5

axes = [ax00,ax01,ax02,ax03,ax04, ax05,ax11,ax12,ax13,ax14]

data_to_plot_R = [CSK_R_disp_rate_1,
                  CSK_R_disp_rate_2,
                  CSK_R_disp_rate_3,
                  CSK_R_disp_rate_4,
                  CSK_R_disp_rate_5,]

data_to_plot_A = [CSK_A_disp_rate_1,
                CSK_A_disp_rate_2,
                CSK_A_disp_rate_3,
                CSK_A_disp_rate_4,
                CSK_A_disp_rate_5,]

data_to_plot_Mag = [np.hypot(CSK_A_disp_rate_1,CSK_R_disp_rate_1),
                    np.hypot(CSK_A_disp_rate_2,CSK_R_disp_rate_2),
                    np.hypot(CSK_A_disp_rate_3,CSK_R_disp_rate_3),
                    np.hypot(CSK_A_disp_rate_4,CSK_R_disp_rate_4),
                    np.hypot(CSK_A_disp_rate_5,CSK_R_disp_rate_5),
                    np.hypot(CSK_A_disp_rate_total,CSK_R_disp_rate_total)]

Lons = [CSK_TS['Longitude'],
        CSK_TS['Longitude'],
        CSK_TS['Longitude'],
        CSK_TS['Longitude'],
        CSK_TS['Longitude'],
        CSK_TS['Longitude'],]


Lats = [CSK_TS['Latitude'],
        CSK_TS['Latitude'],
        CSK_TS['Latitude'],
        CSK_TS['Latitude'],
        CSK_TS['Latitude'],
        CSK_TS['Latitude'],]


SCALE = 100 # cm/m



clims = [(0,0.3),
         (0,15),
         (0,3),
         (0,5),
         (0,5),
         (0,1),]

clims = [(0,3),
         (0,3),
         (0,3),
         (0,3),
         (0,3),
         (0,3),]

delta_T = [CSK_delta_t_1.days,
           CSK_delta_t_2.days,
           CSK_delta_t_3.days,
           CSK_delta_t_4.days,
           CSK_delta_t_5.days,
           CSK_delta_t_total.days,]

cmaps = ['viridis','viridis','viridis', 'viridis', 'viridis', 'viridis',]

labels = string.ascii_uppercase[0:len(axes)]

label_x = [0.03,0.03,0.03,0.03,0.03,
           0.03,0.03,0.03,0.03]
label_y = [0.86,0.86,0.86,0.86,0.86,
           0.86,0.86,0.86,0.86,]

lon_lims = [110.432, 110.45]
lat_lims = [-7.547, -7.535]
grid_size = 1000
disp = 1
minmax = 0
bbox = dict(fc="1.0",alpha=0.5)
for (a,data,lon,lat,label,lab_x,lab_y,cmap,clim,t_diff) in zip(axes,data_to_plot_Mag[0:6],Lons,Lats,labels,label_x,label_y,cmaps,clims,delta_T):
    if disp:
        if minmax:
            vmax = np.max(np.abs(data*t_diff))
            print(vmax)
            plot_data = a.hexbin(lon,lat,C=data*t_diff,gridsize=grid_size,cmap=cmap,vmin=-vmax,vmax=vmax)
        else:
            plot_data = a.hexbin(lon,lat,C=data*t_diff,gridsize=grid_size,cmap=cmap,vmin=clim[0],vmax=clim[1])
    else:
        if minmax:
            vmax = np.max(np.abs(data*SCALE))
            print(vmax)
            plot_data = a.hexbin(lon,lat,C=data*SCALE,gridsize=grid_size,cmap=cmap,vmin=-vmax,vmax=vmax)
        else:
            plot_data = a.hexbin(lon,lat,C=data*SCALE,gridsize=grid_size,cmap=cmap,vmin=clim[0],vmax=clim[1])
    a.plot(coords_L1888[:,0],coords_L1888[:,1],linewidth=2,color='tab:blue',label='L1888')
    a.plot(coords_L1948[:,0],coords_L1948[:,1],linewidth=2,color='tab:cyan',label='L1948')
    a.plot(coords_L1956_1[:,0],coords_L1956_1[:,1],linewidth=2,color='tab:red',label='L1956')
    a.plot(coords_L1956_2[:,0],coords_L1956_2[:,1],linewidth=2,color='tab:red')
    a.plot(coords_L1992[:,0],coords_L1992[:,1],linewidth=2,color='tab:purple',label='L1992')
    a.plot(coords_L1997[:,0],coords_L1997[:,1],linewidth=2,color='tab:olive',label='L1997')
    a.plot(coords_L1998[:,0],coords_L1998[:,1],linewidth=2,color='tab:green',label='L1998')
    a.plot(coords_CRATER[:,0],coords_CRATER[:,1],linewidth=2,color='black',linestyle='--',label='Crater rim')
    a.set_xlim(lon_lims)
    a.set_ylim(lat_lims)
    a.set_aspect('equal', 'box')
    a.add_artist(ScaleBar(sm.plot.get_1deg_dist(),location='lower left'))
    # a.imshow(np.fliplr(np.rot90(image_crop,3)),cmap='Greys_r')
    a.annotate(label,xy=[lab_x,lab_y],xycoords='axes fraction',fontsize=16, bbox=bbox)
    # a.annotate(date.strftime('%Y-%m-%d'),xy=[lab_x+0.0,lab_y-0.795],xycoords='axes fraction',fontsize=16, bbox=bbox)
    cax = add_bottom_cax(a,0.04,0.02)
    cbar = fig.colorbar(plot_data,cax= cax, label='2D displacement magnitude [m]', orientation = 'horizontal', location='bottom')
    a.set_axis_off()
# ax10.set_axis_off()
ax04.legend(bbox_to_anchor=(1.05, 1),
                         loc='upper left', borderaxespad=0., frameon=False)

# fig.subplots_adjust(wspace=0.01,hspace=0.01)




In [49]:
# visualise results

%matplotlib osx
# display image
import string
plt.close('all')

fig = plt.figure(figsize=(15,8))

gs = GridSpec(2, 2, figure=fig)

ax00 = fig.add_subplot(gs[0, 0]) # CSK P1
ax01 = fig.add_subplot(gs[0, 1]) # CSK P2
ax02 = fig.add_subplot(gs[1, 0]) # CSK P3
ax03 = fig.add_subplot(gs[1, 1]) # CSK P4

# ax04 = fig.add_subplot(gs[1, 1]) # CSK P5


# ax10 = fig.add_subplot(gs[1, 0]) # legend?
# ax11 = fig.add_subplot(gs[1, 1]) # TSX P2
# ax12 = fig.add_subplot(gs[1, 2]) # TSX P3
# ax13 = fig.add_subplot(gs[1, 3]) # TSX P4
# ax14 = fig.add_subplot(gs[1, 4]) # TSX P5

axes = [ax00,ax01,ax02,ax03,ax04,ax11,ax12,ax13,ax14]

data_to_plot_R = [TSX_R_disp_rate_2,
                  TSX_R_disp_rate_3,
                  TSX_R_disp_rate_4,
                  TSX_R_disp_rate_5,]

data_to_plot_A = [TSX_A_disp_rate_2,
                  TSX_A_disp_rate_3,
                  TSX_A_disp_rate_4,
                  TSX_A_disp_rate_5,]

data_to_plot_Mag = [np.hypot(TSX_A_disp_rate_2,TSX_R_disp_rate_2),
                    np.hypot(TSX_A_disp_rate_3,TSX_R_disp_rate_3),
                    np.hypot(TSX_A_disp_rate_4,TSX_R_disp_rate_4),
                    np.hypot(TSX_A_disp_rate_5,TSX_R_disp_rate_5),]

Lons = [TSX_TS['Longitude'],
        TSX_TS['Longitude'],
        TSX_TS['Longitude'],
        TSX_TS['Longitude'],]


Lats = [TSX_TS['Latitude'],
        TSX_TS['Latitude'],
        TSX_TS['Latitude'],
        TSX_TS['Latitude'],]


SCALE = 100 # cm/m



clims = [(0,10),
         (0,2),
         (0,7),
         (0,0.5),]

delta_T = [TSX_delta_t_2.days,
           TSX_delta_t_3.days,
           TSX_delta_t_4.days,
           TSX_delta_t_5.days,]

cmaps = ['viridis','viridis','viridis', 'viridis', 'viridis',]

labels = string.ascii_uppercase[0:len(axes)]

label_x = [0.03,0.03,0.03,0.03,0.03,
           0.03,0.03,0.03,0.03]
label_y = [0.86,0.86,0.86,0.86,0.86,
           0.86,0.86,0.86,0.86,]

lon_lims = [110.432, 110.45]
lat_lims = [-7.547, -7.535]
grid_size = 700
disp = 1
minmax = 0
bbox = dict(fc="1.0",alpha=0.5)
for (a,data,lon,lat,label,lab_x,lab_y,cmap,clim,t_diff) in zip(axes,data_to_plot_Mag[0:5],Lons,Lats,labels,label_x,label_y,cmaps,clims,delta_T):
    if disp:
        if minmax:
            vmax = np.max(np.abs(data*t_diff))
            print(vmax)
            plot_data = a.hexbin(lon,lat,C=data*t_diff,gridsize=grid_size,cmap=cmap,vmin=-vmax,vmax=vmax)
        else:
            plot_data = a.hexbin(lon,lat,C=data*t_diff,gridsize=grid_size,cmap=cmap,vmin=clim[0],vmax=clim[1])
    else:
        if minmax:
            vmax = np.max(np.abs(data*SCALE))
            print(vmax)
            plot_data = a.hexbin(lon,lat,C=data*SCALE,gridsize=grid_size,cmap=cmap,vmin=-vmax,vmax=vmax)
        else:
            plot_data = a.hexbin(lon,lat,C=data*SCALE,gridsize=grid_size,cmap=cmap,vmin=clim[0],vmax=clim[1])
    a.plot(coords_L1888[:,0],coords_L1888[:,1],linewidth=2,color='tab:blue',label='L1888')
    a.plot(coords_L1948[:,0],coords_L1948[:,1],linewidth=2,color='tab:cyan',label='L1948')
    a.plot(coords_L1956_1[:,0],coords_L1956_1[:,1],linewidth=2,color='tab:red',label='L1956')
    a.plot(coords_L1956_2[:,0],coords_L1956_2[:,1],linewidth=2,color='tab:red')
    a.plot(coords_L1992[:,0],coords_L1992[:,1],linewidth=2,color='tab:purple',label='L1992')
    a.plot(coords_L1997[:,0],coords_L1997[:,1],linewidth=2,color='tab:olive',label='L1997')
    a.plot(coords_L1998[:,0],coords_L1998[:,1],linewidth=2,color='tab:green',label='L1998')
    a.plot(coords_CRATER[:,0],coords_CRATER[:,1],linewidth=2,color='black',linestyle='--',label='Crater rim')
    a.set_xlim(lon_lims)
    a.set_ylim(lat_lims)
    a.set_aspect('equal', 'box')
    a.add_artist(ScaleBar(sm.plot.get_1deg_dist(),location='lower left'))
    # a.imshow(np.fliplr(np.rot90(image_crop,3)),cmap='Greys_r')
    a.annotate(label,xy=[lab_x,lab_y],xycoords='axes fraction',fontsize=16, bbox=bbox)
    # a.annotate(date.strftime('%Y-%m-%d'),xy=[lab_x+0.0,lab_y-0.795],xycoords='axes fraction',fontsize=16, bbox=bbox)
    cax = add_bottom_cax(a,0.0,0.02)
    cbar = fig.colorbar(plot_data,cax= cax, label='2D displacement magnitude [m]', orientation = 'horizontal', location='bottom')
    a.set_axis_off()
# ax10.set_axis_off()
ax03.legend(bbox_to_anchor=(1.05, 1),
                         loc='upper left', borderaxespad=0., frameon=False)

# fig.subplots_adjust(wspace=0.01,hspace=0.01)


